# Поиск токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Модель должна иметь метрику качества *F1* не меньше 0.75. 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

Загрузим всё необходимое для работы

In [1]:
import pandas as pd

import numpy as np

import re 

from pymystem3 import Mystem

import nltk

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score

from nltk.stem import WordNetLemmatizer

In [2]:
df_toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df_toxic_comments

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


Первое, что сделаем после загрузки в данном случае это посмотрим на баланс классов. 

In [4]:
df_toxic_comments['toxic'].mean() * 100

10.167887648758233

Далеко не 1:1. Учтём это при построении моделей.

In [5]:
target = df_toxic_comments['toxic']

In [6]:
corpus = df_toxic_comments['text']

Разобьём данные на обучающую и тестовую выборку.

In [7]:
corpus_train, corpus_test, target_train, target_test  = train_test_split(corpus, 
                                                           target, 
                                                           random_state = 12345, 
                                                           test_size = 0.1)

In [8]:
corpus_train = corpus_train.values.astype('U')

In [9]:
corpus_test = corpus_test.values.astype('U')

Напишем функцию для подготовки текстов, выполняющую:
1. Лемматизацию текста с помощью библиотеке `NLTK`;
2. Устранение лишних символов и пробелов.

Для наглядности, выведем пару предложение корпуса до и после применения функции.

In [10]:
corpus_train[:2]

array(["There is something wrong with you, Madam! \n\nMadam, stop all of this nonsence on the ABC Kids page, I believe that Disney's One Saturday Morning launched in 1996, not 1997, and I'm thinking Mater's Tall Tales was not on ABC Kids, so cut this crap out, Darby, or I'm gonna kill you.   12:09 6 July 2012 (UTC)",
       'Amyas Godfrey\nThe page has come under scrutiny as of late. How can we fix it so it is more in line with Wiki guidelines? Should it stay or should it go?'],
      dtype='<U5000')

In [11]:
def preparation(text):
    for i in range (0, len(text)):
        text[i] = re.sub(r'[^a-zA-Z]', ' ', text[i])  # Устранение лишних символов
        text[i] = " ".join(text[i].split())           # Устранение лишних пробелов
        w = WordNetLemmatizer() 
        lemmas = w.lemmatize(text[i],'v')
        text[i] = "".join(str(x) for x in lemmas)     # Лемматизация

In [12]:
preparation(corpus_train)

In [13]:
corpus_train[:2]

array(['There is something wrong with you Madam Madam stop all of this nonsence on the ABC Kids page I believe that Disney s One Saturday Morning launched in not and I m thinking Mater s Tall Tales was not on ABC Kids so cut this crap out Darby or I m gonna kill you July UTC',
       'Amyas Godfrey The page has come under scrutiny as of late How can we fix it so it is more in line with Wiki guidelines Should it stay or should it go'],
      dtype='<U5000')

Разница налицо.

In [14]:
preparation(corpus_test)

Рассчитаем `TF-IDF` для каждой выборки.

In [ ]:
nltk.download('stopwords') 

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words = stop_words)

In [ ]:
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

In [ ]:
tf_idf_test = count_tf_idf.transform(corpus_test)

## Обучение

Итак, для классификации комментариев используем следующие виды моделей:
1. `DecisionTreeClassifier`;
2. `RandomForestClassifier`;
3. `LogisticRegression`.

*`DecisionTreeClassifier`*

В данном случае исследуем результат при изменении гиперпараметра `max_depth`.

In [ ]:
list_f1_score_tree = []
max_depth_tree = []

for depth in range(1,21): 
    
    model = DecisionTreeClassifier(random_state=12345, 
                                   max_depth = depth, 
                                   class_weight = 'balanced')
    
    model.fit(tf_idf_train,target_train)
    predictions = model.predict(tf_idf_test)
    
    f1_score_1 = f1_score(target_test, predictions) 
    
    list_f1_score_tree.append(f1_score_1)
    max_depth_tree.append(depth)

columns = {
            "max_depth_tree":np.array(max_depth_tree),
            "f1_score_tree": np.array(list_f1_score_tree),
          }

df_max_f1_score_tree = pd.DataFrame(columns)

df_max_f1_score_tree.query('f1_score_tree == f1_score_tree.max()')

*`RandomForestClassifier`*

Для леса исследуем следующие гиперпараметры: 
1. `n_estimators`
2. `max_depth`

In [ ]:
list_f1_forest = []
n_estimators_forest = []
max_depth_forest = []

for est in range(100,500,100):
    
    for depth in range(1,20): 
        
        model = RandomForestClassifier(random_state=12345,
                                      n_estimators = est,
                                      max_depth = depth, 
                                      class_weight = 'balanced')
        
        model.fit(tf_idf_train,target_train)
        predictions = model.predict(tf_idf_test)
        f1_score_2 = f1_score(target_test, predictions) 
    
        list_f1_forest.append(f1_score_2)
        n_estimators_forest.append(est)
        max_depth_forest.append(depth)

columns = {
            "max_depth_forest":np.array(max_depth_forest),
            "n_estimators_forest":np.array(n_estimators_forest),
            "f1_forest": np.array(list_f1_forest),
          }

df_max_f1_forest = pd.DataFrame(columns)

df_max_f1_forest.query('f1_forest == f1_forest.max()')

*`LogisticRegression`*

Прежде, чем определиться с подбором гиперпараметров, посмотрим за какое время и какое значение метрики `f1_score` можно получить на стандратных настройках для логистической регрессии.

In [ ]:
%%time 

model = LogisticRegression(random_state=12345, class_weight = 'balanced')
model.fit(tf_idf_train,target_train)
predictions = model.predict(tf_idf_test)
f1_score_3 = f1_score(target_test, predictions)
print(f1_score_3)

В целом, это уже выше необходимого значения. Поэтому подойдём к вопросу исследования гиперпараметров чисто символически. Чтобы не растягивать расчёт на часы, выберем только 2 из них:

1. `penalty`
2. `C`

In [ ]:
%%time

list_f1_logistic = []
list_penalty = []
list_c = []

for penalty in ['l1','l2']:
    
    for c in range(1,21,1):
        
        model = LogisticRegression(random_state=12345,
                                              penalty = penalty,
                                              C = c, 
                                              class_weight = 'balanced')
        
        model.fit(tf_idf_train,target_train)
        predictions = model.predict(tf_idf_test)
        f1_score_4 = f1_score(target_test, predictions)
        
        list_f1_logistic.append(f1_score_4)
        list_penalty.append(penalty)
        list_c.append(c)
    
columns = {
            "penalty":np.array(list_penalty),
            "c":np.array(list_c),
            "f1_logistic": np.array(list_f1_logistic),
          }
                

df_max_f1_logistic = pd.DataFrame(columns)

df_max_f1_logistic.query('f1_logistic == f1_logistic.max()')        

## Выводы

Итак, по итогам проведённых исследований получили следующие результаты: 

In [ ]:
names = ['DecisionTreeClassifier','RandomForestClassifier','LogisticRegression']
column = {"f1_score": np.array([0.60, 0.41, 0.78])}
final_f1_score = pd.DataFrame(column, index = names) 

In [ ]:
final_f1_score

Таким образом, из преведённой выше таблицы хорошо видно, что лучший прогноз на предмет тональности комментария мы сможем получить с помощью модели логистической регрессии `LogisticRegression` с гиперпараметрами `penalty` = `l2`, `c` = `7`. В свою очередь решить проблему дисбаланса классов позволяет гиперпараметр `class_weight = 'balanced'`.Для лемматизации используем библиотеку `NLTK`. Значение метрики `f1_score` составит в этом случае `0.78`. 